In [1]:
import napari
from skimage.io import imread
import numpy as np
import pandas as pd
import time
from napari_clusters_plotter._utilities import get_nice_colormap
from vispy.color import Color
import pyclesperanto_prototype as cle

regprops = pd.read_csv('table_with_prediction.csv')
cluster_id = regprops['MANUAL_CLUSTER_ID'].to_numpy()
labels_list = regprops['label'].to_numpy()

label_image = imread(r"C:\Users\ryans\Documents\output data (big)\Finsterwalde Gastrulation Labels (new timeframe)\workflow t = 49837.5s.tif")

colors =get_nice_colormap()
cmap = [Color(hex_name).RGBA.astype("float") / 255 for hex_name in colors]
cmap_dict = {
    int(prediction + 1): (
        cmap[int(prediction) % len(cmap)]
        if prediction >= 0
        else [0, 0, 0, 0]
    )
    for prediction in cluster_id
}
# take care of background label
cmap_dict[0] = [0, 0, 0, 0]

viewer = napari.Viewer()
viewer.add_labels(label_image)

def generate_cluster_image_cle(label_image, predictionlist):
    """
    Returns a label image where each label value corresponds
    to the cluster identity defined by the predictionlist.
    it is assumed that len(predictionlist) == max(label_image)

    Parameters
    ----------
    label_image: ndarray or dask array
        Label image used for cluster predictions
    predictionlist: array
        Array containing cluster identities for each label
    """
    # reforming the prediction list this is done to account
    # for cluster labels that start at 0 conviniently hdbscan
    # labelling starts at -1 for noise, removing these from
    # the labels
    predictionlist = np.array(predictionlist) + 1
    predictionlist_new = np.asarray([0] + predictionlist)

    # loading data into gpu
    gpu_labelimage = cle.push(label_image)

    # generation of cluster label image
    parametric_image = cle.replace_intensities(gpu_labelimage, predictionlist_new)
    gpu_labelimage = None

    # retrieving the gpu image
    output = cle.pull(parametric_image).astype("uint32")
    parametric_image = None

    return output

def generate_cluster_image(label_image, predictionlist):
    """
    Returns a label image where each label value corresponds
    to the cluster identity defined by the predictionlist.
    it is assumed that len(predictionlist) == max(label_image)

    Parameters
    ----------
    label_image: ndarray or dask array
        Label image used for cluster predictions
    predictionlist: array
        Array containing cluster identities for each label
    """
    # reforming the prediction list this is done to account
    # for cluster labels that start at 0 conviniently hdbscan
    # labelling starts at -1 for noise, removing these from
    # the labels
    predictionlist_new = np.array(predictionlist) + 1
    predictionlist_new = np.insert(predictionlist_new, 0, 0)

    # loading data into gpu
    clelist = cle.push(predictionlist_new)
    gpu_labelimage = cle.push(label_image)

    # generation of cluster label image
    parametric_image = cle.replace_intensities(gpu_labelimage, clelist)
    gpu_labelimage = None
    clelist = None

    # retrieving the gpu image
    output = cle.pull(parametric_image).astype("uint32")
    parametric_image = None

    return output

def generate_cluster_image_np(label_image, predictionlist):
    predictionlist_new = np.array(predictionlist) + 1
    predictionlist_new = np.insert(predictionlist_new, 0, 0)
    return np.take(predictionlist_new, label_image)


def relabel_numpy(image, measurements):
    return np.take(np.array([0] + measurements), image)

def generate_label_to_cluster_color_mapping(label_list, predictionlist, colormap_dict):
    predictionlist_new = np.array(predictionlist) + 1
    mapping = {0: [0,0,0,0]}
    for label,prediction in zip(label_list,predictionlist_new):
        mapping[label] = colormap_dict[prediction]
        
    return mapping

C:\Users\ryans\Anaconda3\envs\ncp-dev\lib\site-packages\napari_tools_menu\__init__.py:165: FutureWarning: Public access to Window.qt_viewer is deprecated and will be removed in
v0.5.0. It is considered an "implementation detail" of the napari
application, not part of the napari viewer model. If your use case
requires access to qt_viewer, please open an issue to discuss.
  self.tools_menu = ToolsMenu(self, self.qt_viewer.viewer)


## Pyclesperanto time

In [2]:
starttime = time.process_time_ns()
cl_image = generate_cluster_image(label_image,predictionlist=cluster_id)

viewer.add_labels(cl_image, color=cmap_dict)

endtime = time.process_time_ns()
cle_time = endtime-starttime
print(f'pyclesperanto took {cle_time/1_000_000} ms')

pyclesperanto took 828.125 ms


## Numpy Time

In [3]:
starttime = time.process_time_ns()
np_image = generate_cluster_image_np(label_image,predictionlist=cluster_id)

viewer.add_labels(np_image, color=cmap_dict)

endtime = time.process_time_ns()
np_time = endtime - starttime

print(f'numpy took {np_time/1_000_000} ms')


numpy took 968.75 ms


## Colormap Time
### with adding a layer

In [4]:
starttime = time.process_time_ns()

mapping_layer = viewer.add_labels(label_image,name = 'cluster id mapping')
mapping = generate_label_to_cluster_color_mapping(
    label_list=labels_list, 
    predictionlist= cluster_id, 
    colormap_dict= cmap_dict
)
mapping_layer.color = mapping

endtime = time.process_time_ns()
mapping_time = endtime - starttime

print(f'mapping took {mapping_time/1_000_000} ms')

mapping took 187.5 ms
